## Introduction
This notebook focuses on the initial cleaning of the diabetic patient dataset to ensure it is suitable for machine learning tasks. The raw data contains inconsistencies, missing values, and irrelevant features that could negatively impact model performance. Key steps include handling missing values (represented as “?”), dropping features with excessive null values, and converting categorical columns into appropriate formats. Unique identifiers that offered no predictive power were removed, and additional contextual data from a mapping file was merged to enhance interpretability. This cleaning phase ensures a solid foundation for downstream tasks like feature engineering and modeling.



In [ ]:
# Importing all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, silhouette_score
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import joblib

# Importing additional libraries for advanced tasks
try:
    import plotly.express as px
    import imblearn
    from feature_engine import imputation
    import xgboost as xgb
    import lightgbm as lgb
    import catboost as cb
    import missingno as msno
    import statsmodels.api as sm
except ImportError:
    print("Optional libraries not installed. Install them if needed.")

# All libraries are now ready to use
print("Libraries imported successfully!")

# Importing the warnings library to ignore warnings
import warnings
warnings.filterwarnings('ignore')

print("Warnings will now be ignored!")

In [4]:
def load_data(file_path):
    """
    Function to load a dataset from a specified file path.
    
    Parameters:
    - file_path (str): The file path of the dataset.
    
    Returns:
    - data (DataFrame): Loaded dataset as a pandas DataFrame.
    """
    try:
        data = pd.read_csv(file_path)
        print(f"Data loaded successfully! Shape: {data.shape}")
        return data
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
    except pd.errors.ParserError:
        print("Error: File could not be parsed. Please check the file format.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

#usage 
diabetes_dataset = load_data('diabetic_data.csv')
IDS_mapping_dataset = load_data('IDS_mapping.csv')

Data loaded successfully! Shape: (101766, 50)
Data loaded successfully! Shape: (67, 2)


## Data Cleaning

In [10]:
def inspect_data(data, name):
    """
    Function to inspect the structure, column names, data types, and summary statistics of a dataset.
    
    Parameters:
    - data (DataFrame): The dataset to be inspected.
    - name (str): A name or label for the dataset.
    """
    print(f"\n--- Inspection of {name} Dataset ---\n")
    
    # Display the first few rows of the dataset
    print("First 5 Rows:")
    print(data.head())
    print("\n")
    
    # Display the shape of the dataset
    print(f"Shape of the dataset: {data.shape}")
    
    # Display column names and data types
    print("\nColumn Names and Data Types:")
    print(data.dtypes)
    print("\n")
    
    # Display summary statistics for numerical columns
    print("Summary Statistics (Numerical Columns):")
    print(data.describe())
    print("\n")
    
    # Display summary for categorical columns
    print("Value Counts for Categorical Columns:")
    categorical_cols = data.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        print(f"\n{col} Value Counts:")
        print(data[col].value_counts())
    


In [12]:
# Usage
inspect_data(diabetes_dataset, "Diabetes Dataset")



--- Inspection of Diabetes Dataset Dataset ---

First 5 Rows:
   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)      ?   
1        149190     55629189        Caucasian  Female  [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female  [20-30)      ?   
3        500364     82442376        Caucasian    Male  [30-40)      ?   
4         16680     42519267        Caucasian    Male  [40-50)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0 

### Column Names and Data Types:
- The dataset includes a mix of numeric (`int64`) and categorical (`object`) features.
- **Potential issues:**
  - Columns like `race` and `weight` contain categorical data with unclear or missing values (e.g., `?`).
  - Some numeric identifiers (e.g., `admission_type_id`) should be mapped to meaningful labels.

### Summary Statistics (Numerical Columns):
- **Basic statistics for numerical columns like `time_in_hospital`:**
  - Mean hospital stay: ~4.4 days.
  - Max stay: 14 days.
- Columns like `number_emergency` and `number_inpatient` have large ranges, suggesting potential outliers.

### Value Counts for Categorical Columns:
- **Key observations:**
  - **`race`:** Most patients are Caucasian, but there are 2273 entries with missing values (`?`).
  - **`gender`:** Balanced distribution between Male and Female, with 3 invalid entries (`Unknown/Invalid`).
  - **`age`:** Older age groups (e.g., `[60-70)` and `[70-80)`) dominate.
  - **`weight`:** Most entries (98569) are missing (`?`), indicating a need for imputation or exclusion.
  - **`readmitted`:** Target variable with three categories (`NO`, `<30`, `>30`).

In [14]:
inspect_data(IDS_mapping_dataset, "IDS Mapping Dataset")


--- Inspection of IDS Mapping Dataset Dataset ---

First 5 Rows:
  admission_type_id    description
0                 1      Emergency
1                 2         Urgent
2                 3       Elective
3                 4        Newborn
4                 5  Not Available


Shape of the dataset: (67, 2)

Column Names and Data Types:
admission_type_id    object
description          object
dtype: object


Summary Statistics (Numerical Columns):
       admission_type_id      description
count                 65               62
unique                32               58
top                    1  Unknown/Invalid
freq                   3                2


Value Counts for Categorical Columns:

admission_type_id Value Counts:
admission_type_id
1                           3
6                           3
8                           3
7                           3
2                           3
5                           3
4                           3
3                           3
18       

In [18]:
def identify_missing_values(data, dataset_name):
    """
    Function to identify missing values, placeholders, and inconsistencies in a dataset.
    
    Parameters:
    - data (DataFrame): The dataset to be inspected.
    - dataset_name (str): Name of the dataset for labeling output.
    """
    print(f"\n--- Missing Values Analysis for {dataset_name} ---\n")
    
    # 1. Check for NaN values in the dataset
    missing_nan = data.isnull().sum()
    print("Missing (NaN) Values by Column:")
    print(missing_nan[missing_nan > 0])
    print("\n")
    
    # 2. Check for placeholder values (e.g., "?")
    placeholder_cols = []
    for col in data.columns:
        if data[col].dtype == 'object':
            if data[col].str.contains(r'\?').any():
                placeholder_cols.append(col)
    
    print("Columns with Placeholder Values ('?'):")
    if placeholder_cols:
        for col in placeholder_cols:
            print(f"- {col}: {data[col].value_counts().get('?', 0)} placeholder values")
    else:
        print("No columns with placeholder values detected.")
    print("\n")
    
    # 3. Check for empty strings in categorical columns
    empty_string_cols = []
    for col in data.columns:
        if data[col].dtype == 'object':
            if (data[col] == "").any():
                empty_string_cols.append(col)
    
    print("Columns with Empty String Values:")
    if empty_string_cols:
        for col in empty_string_cols:
            print(f"- {col}: {data[col].value_counts().get('', 0)} empty string values")
    else:
        print("No columns with empty string values detected.")
    print("\n")

# Usage
identify_missing_values(diabetes_dataset, "Diabetes Dataset")
identify_missing_values(IDS_mapping_dataset, "IDS Mapping Dataset")


--- Missing Values Analysis for Diabetes Dataset ---

Missing (NaN) Values by Column:
max_glu_serum    96420
A1Cresult        84748
dtype: int64


Columns with Placeholder Values ('?'):
- race: 2273 placeholder values
- weight: 98569 placeholder values
- payer_code: 40256 placeholder values
- medical_specialty: 49949 placeholder values
- diag_1: 21 placeholder values
- diag_2: 358 placeholder values
- diag_3: 1423 placeholder values


Columns with Empty String Values:
No columns with empty string values detected.



--- Missing Values Analysis for IDS Mapping Dataset ---

Missing (NaN) Values by Column:
admission_type_id    2
description          5
dtype: int64


Columns with Placeholder Values ('?'):
No columns with placeholder values detected.


Columns with Empty String Values:
No columns with empty string values detected.




In [20]:
def clean_placeholder_values(data, categorical_columns, numerical_columns):
    """
    Function to replace placeholder values in categorical and numerical columns.

    Parameters:
    - data (DataFrame): The dataset to clean.
    - categorical_columns (list): List of categorical columns to clean.
    - numerical_columns (list): List of numerical columns to clean.

    Returns:
    - DataFrame: Cleaned dataset.
    """
    print("\n--- Cleaning Placeholder Values ---\n")
    
    # 1. Replace '?' with "Unknown" for categorical columns
    for col in categorical_columns:
        if col in data.columns:
            if data[col].dtype == 'object':
                original_count = data[col].value_counts().get('?', 0)
                data[col] = data[col].replace('?', 'Unknown')
                print(f"Replaced {original_count} '?' values in column '{col}' with 'Unknown'.")

    # 2. Handle numerical columns with '?' placeholders
    for col in numerical_columns:
        if col in data.columns:
            # Replace '?' with NaN for imputation
            data[col] = pd.to_numeric(data[col], errors='coerce')
            if data[col].isnull().sum() > 0:
                # Impute with mean for simplicity (could be changed to median or mode)
                mean_value = data[col].mean()
                data[col].fillna(mean_value, inplace=True)
                print(f"Imputed missing values in numerical column '{col}' with mean: {mean_value:.2f}.")

    # 3. Address specific placeholders
    # Replace 'Not Available' or similar values with 'Unknown'
    placeholders = ['Not Available', 'Unknown/Invalid']
    for col in categorical_columns:
        if col in data.columns:
            for placeholder in placeholders:
                original_count = data[col].value_counts().get(placeholder, 0)
                data[col] = data[col].replace(placeholder, 'Unknown')
                if original_count > 0:
                    print(f"Replaced {original_count} '{placeholder}' values in column '{col}' with 'Unknown'.")

    return data

# Specify categorical and numerical columns to clean
categorical_columns = ['weight', 'payer_code', 'medical_specialty', 'race', 'gender', 'age']
numerical_columns = ['time_in_hospital', 'num_lab_procedures', 'num_medications', 'number_inpatient']

# Clean the diabetes dataset
diabetes_dataset_cleaned = clean_placeholder_values(diabetes_dataset, categorical_columns, numerical_columns)


--- Cleaning Placeholder Values ---

Replaced 98569 '?' values in column 'weight' with 'Unknown'.
Replaced 40256 '?' values in column 'payer_code' with 'Unknown'.
Replaced 49949 '?' values in column 'medical_specialty' with 'Unknown'.
Replaced 2273 '?' values in column 'race' with 'Unknown'.
Replaced 0 '?' values in column 'gender' with 'Unknown'.
Replaced 0 '?' values in column 'age' with 'Unknown'.
Replaced 3 'Unknown/Invalid' values in column 'gender' with 'Unknown'.


## Data Type Standardization

In [23]:
# Clean the IDS_mapping dataset
def clean_ids_mapping(data):
    """
    Function to handle missing description values in the IDS_mapping dataset.

    Parameters:
    - data (DataFrame): The IDS_mapping dataset.

    Returns:
    - DataFrame: Cleaned IDS_mapping dataset.
    """
    print("\n--- Cleaning IDS_mapping Dataset ---\n")
    
    # Count missing values in 'description' column
    missing_count = data['description'].isnull().sum()
    
    if missing_count > 0:
        print(f"Found {missing_count} missing values in 'description' column.")
        
        # Replace missing values with 'Unknown'
        data['description'].fillna('Unknown', inplace=True)
        print(f"Replaced missing values in 'description' column with 'Unknown'.")
    else:
        print("No missing values found in 'description' column.")

    return data

IDS_mapping_cleaned = clean_ids_mapping(IDS_mapping_dataset)
IDS_mapping_cleaned.head()


--- Cleaning IDS_mapping Dataset ---

Found 5 missing values in 'description' column.
Replaced missing values in 'description' column with 'Unknown'.


,admission_type_id,description
0,1,Emergency
1,2,Urgent
2,3,Elective
3,4,Newborn
4,5,Not Available


In [33]:
def clean_and_standardize_ids_mapping(data):
    """
    Function to clean and standardize the IDS_mapping dataset.
    - Fills missing values in 'description' with 'Unknown'.
    - Converts 'admission_type_id' to string for validation.
    - Removes invalid entries from 'admission_type_id'.
    - Converts 'admission_type_id' to integers.

    Parameters:
    - data (DataFrame): The IDS_mapping dataset.

    Returns:
    - DataFrame: Cleaned and standardized IDS_mapping dataset.
    """
    print("\n--- Cleaning and Standardizing IDS_mapping Dataset ---\n")
    
    # Fill missing values in 'description'
    data['description'].fillna('Unknown', inplace=True)

    # Ensure 'admission_type_id' is string for validation
    data['admission_type_id'] = data['admission_type_id'].astype(str)

    # Identify and remove invalid entries in 'admission_type_id'
    invalid_entries = data[~data['admission_type_id'].str.isdigit()]
    if not invalid_entries.empty:
        print(f"Found invalid entries in 'admission_type_id':\n{invalid_entries}")
        # Drop invalid rows
        data = data[data['admission_type_id'].str.isdigit()]

    # Convert 'admission_type_id' back to integer
    data['admission_type_id'] = data['admission_type_id'].astype(int)
    print("Converted 'admission_type_id' to integer type.")
    
    return data

# Apply the cleaning and standardization
IDS_mapping_cleaned_standardized = clean_and_standardize_ids_mapping(IDS_mapping_dataset)

# Display the cleaned and standardized dataset
IDS_mapping_cleaned_standardized.head()


--- Cleaning and Standardizing IDS_mapping Dataset ---

Found invalid entries in 'admission_type_id':
           admission_type_id  description
8                         -1      Unknown
9   discharge_disposition_id  description
40                        -1      Unknown
41       admission_source_id  description
Converted 'admission_type_id' to integer type.


,admission_type_id,description
0,1,Emergency
1,2,Urgent
2,3,Elective
3,4,Newborn
4,5,Not Available


In [27]:
def standardize_diabetic_data(data):
    """
    Function to standardize data types for diabetic_data.

    Parameters:
    - data (DataFrame): The diabetic_data dataset.

    Returns:
    - DataFrame: Standardized diabetic_data dataset.
    """
    print("\n--- Standardizing diabetic_data Dataset ---\n")

    # 3.1 Convert encounter_id and patient_nbr to string types
    data['encounter_id'] = data['encounter_id'].astype(str)
    data['patient_nbr'] = data['patient_nbr'].astype(str)
    print("Converted 'encounter_id' and 'patient_nbr' to string type.")

    # 3.2 Standardize age ranges to numeric intervals
    age_mapping = {
        '[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35,
        '[40-50)': 45, '[50-60)': 55, '[60-70)': 65, '[70-80)': 75,
        '[80-90)': 85, '[90-100)': 95
    }
    data['age'] = data['age'].map(age_mapping)
    print("Standardized 'age' ranges to numeric intervals.")

    # 3.3 Ensure ID columns are integers
    id_columns = ['admission_type_id', 'discharge_disposition_id', 'admission_source_id']
    for col in id_columns:
        data[col] = data[col].astype(int)
    print("Converted 'admission_type_id', 'discharge_disposition_id', and 'admission_source_id' to integer type.")

    return data

# Apply standardization to diabetic_data
diabetes_dataset_standardized = standardize_diabetic_data(diabetes_dataset)
diabetes_dataset_standardized.head()



--- Standardizing diabetic_data Dataset ---

Converted 'encounter_id' and 'patient_nbr' to string type.
Standardized 'age' ranges to numeric intervals.
Converted 'admission_type_id', 'discharge_disposition_id', and 'admission_source_id' to integer type.


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,5,Unknown,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,15,Unknown,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,25,Unknown,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,35,Unknown,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,45,Unknown,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [37]:
def inspect_and_clean_ids_mapping(data):
    """
    Function to inspect and clean the IDS_mapping dataset.
    - Inspects unique values in 'admission_type_id'.
    - Removes invalid entries.
    - Converts 'admission_type_id' to integer after cleaning.

    Parameters:
    - data (DataFrame): The IDS_mapping dataset.

    Returns:
    - DataFrame: Cleaned IDS_mapping dataset.
    """
    print("\n--- Inspecting and Cleaning IDS_mapping Dataset ---\n")
    
    # Inspect unique values in 'admission_type_id'
    print("Unique values in 'admission_type_id':")
    print(data['admission_type_id'].unique())
    
    # Remove invalid entries (non-numeric)
    valid_entries = data['admission_type_id'].apply(lambda x: str(x).isdigit())
    invalid_entries = data[~valid_entries]
    if not invalid_entries.empty:
        print("\nFound invalid entries in 'admission_type_id':")
        print(invalid_entries)
        # Drop invalid rows
        data = data[valid_entries]

    # Convert 'admission_type_id' to integer
    data['admission_type_id'] = data['admission_type_id'].astype(int)
    print("\nConverted 'admission_type_id' to integer type.")
    
    return data

# Apply the cleaning and inspection
IDS_mapping_cleaned = inspect_and_clean_ids_mapping(IDS_mapping_dataset)

# Display the cleaned dataset
IDS_mapping_cleaned.head()


--- Inspecting and Cleaning IDS_mapping Dataset ---

Unique values in 'admission_type_id':
['1' '2' '3' '4' '5' '6' '7' '8' '-1' 'discharge_disposition_id' '9' '10'
 '11' '12' '13' '14' '15' '16' '17' '18' '19' '20' '21' '22' '23' '24'
 '25' '26' '30' '27' '28' '29' 'admission_source_id']

Found invalid entries in 'admission_type_id':
           admission_type_id  description
8                         -1      Unknown
9   discharge_disposition_id  description
40                        -1      Unknown
41       admission_source_id  description

Converted 'admission_type_id' to integer type.


,admission_type_id,description
0,1,Emergency
1,2,Urgent
2,3,Elective
3,4,Newborn
4,5,Not Available


In [39]:
def map_ids_to_descriptions(data, mapping_data, id_column, description_column, target_column_name):
    """
    Function to map ID columns in diabetic_data to their corresponding descriptions from IDS_mapping.

    Parameters:
    - data (DataFrame): The diabetic_data dataset.
    - mapping_data (DataFrame): The IDS_mapping dataset.
    - id_column (str): The column in diabetic_data containing numeric IDs.
    - description_column (str): The column in IDS_mapping containing descriptions.
    - target_column_name (str): The name of the new column in diabetic_data for mapped descriptions.

    Returns:
    - DataFrame: The diabetic_data dataset with the new descriptive column.
    """
    # Create a dictionary for mapping
    mapping_dict = mapping_data.set_index(id_column)[description_column].to_dict()

    # Map the descriptions to the target column
    data[target_column_name] = data[id_column].map(mapping_dict)

    print(f"Mapped '{id_column}' to '{target_column_name}' using IDS_mapping.")
    return data

# 4.1 Map admission_type_id to descriptions
diabetes_dataset_mapped = map_ids_to_descriptions(
    diabetes_dataset_standardized,
    IDS_mapping_cleaned,
    id_column='admission_type_id',
    description_column='description',
    target_column_name='admission_type_description'
)

# 4.2 Verify mappings for discharge_disposition_id and admission_source_id
# If no mapping is provided, identify unique values for further mapping
discharge_disposition_unique = diabetes_dataset_mapped['discharge_disposition_id'].unique()
admission_source_unique = diabetes_dataset_mapped['admission_source_id'].unique()

# Display unique values for these IDs
print("\nUnique values in 'discharge_disposition_id':")
print(discharge_disposition_unique)

print("\nUnique values in 'admission_source_id':")
print(admission_source_unique)

# Preview the updated dataset
diabetes_dataset_mapped.head()

Mapped 'admission_type_id' to 'admission_type_description' using IDS_mapping.

Unique values in 'discharge_disposition_id':
[25  1  3  6  2  5 11  7 10  4 14 18  8 13 12 16 17 22 23  9 20 15 24 28
 19 27]

Unique values in 'admission_source_id':
[ 1  7  2  4  5  6 20  3 17  8  9 14 10 22 11 25 13]


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admission_type_description
0,2278392,8222157,Caucasian,Female,5,Unknown,6,25,1,1,...,No,No,No,No,No,No,No,No,NO,Transfer from another health care facility
1,149190,55629189,Caucasian,Female,15,Unknown,1,1,7,3,...,Up,No,No,No,No,No,Ch,Yes,>30,Physician Referral
2,64410,86047875,AfricanAmerican,Female,25,Unknown,1,1,7,2,...,No,No,No,No,No,No,No,Yes,NO,Physician Referral
3,500364,82442376,Caucasian,Male,35,Unknown,1,1,7,2,...,Up,No,No,No,No,No,Ch,Yes,NO,Physician Referral
4,16680,42519267,Caucasian,Male,45,Unknown,1,1,7,1,...,Steady,No,No,No,No,No,Ch,Yes,NO,Physician Referral


In [41]:
def handle_duplicates(data):
    """
    Function to address duplicates in diabetic_data.
    - Removes duplicate rows based on the primary key 'encounter_id'.
    - Checks for duplicate entries based on 'patient_nbr' and ensures they represent unique visits.

    Parameters:
    - data (DataFrame): The diabetic_data dataset.

    Returns:
    - DataFrame: Deduplicated diabetic_data dataset.
    """
    print("\n--- Addressing Duplicates in diabetic_data ---\n")
    
    # 5.1 Check and remove duplicates based on 'encounter_id'
    duplicate_encounter_ids = data.duplicated(subset=['encounter_id']).sum()
    print(f"Found {duplicate_encounter_ids} duplicate rows based on 'encounter_id'.")
    data = data.drop_duplicates(subset=['encounter_id'])
    print("Removed duplicates based on 'encounter_id'.")
    
    # 5.2 Check for duplicate entries in 'patient_nbr'
    patient_duplicate_counts = data.duplicated(subset=['patient_nbr']).sum()
    print(f"Found {patient_duplicate_counts} patients with multiple entries in 'patient_nbr'.")

    # Investigate if these represent unique visits or duplicates
    duplicated_patients = data[data.duplicated(subset=['patient_nbr'], keep=False)]
    print(f"Duplicated patient records:\n{duplicated_patients.head()}")
    
    return data

# Apply the function to handle duplicates
diabetes_dataset_deduplicated = handle_duplicates(diabetes_dataset_mapped)

# Display the deduplicated dataset
diabetes_dataset_deduplicated.head()


--- Addressing Duplicates in diabetic_data ---

Found 0 duplicate rows based on 'encounter_id'.
Removed duplicates based on 'encounter_id'.
Found 30248 patients with multiple entries in 'patient_nbr'.
Duplicated patient records:
   encounter_id patient_nbr       race  gender  age   weight  \
12        40926    85504905  Caucasian  Female   45  Unknown   
27       248916   115196778  Caucasian  Female   55  Unknown   
28       250872    41606064  Caucasian    Male   25  Unknown   
32       260166    80845353  Caucasian  Female   75  Unknown   
33       293058   114715242  Caucasian    Male   65  Unknown   

    admission_type_id  discharge_disposition_id  admission_source_id  \
12                  1                         3                    7   
27                  1                         1                    1   
28                  2                         1                    2   
32                  1                         1                    7   
33                  2    

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admission_type_description
0,2278392,8222157,Caucasian,Female,5,Unknown,6,25,1,1,...,No,No,No,No,No,No,No,No,NO,Transfer from another health care facility
1,149190,55629189,Caucasian,Female,15,Unknown,1,1,7,3,...,Up,No,No,No,No,No,Ch,Yes,>30,Physician Referral
2,64410,86047875,AfricanAmerican,Female,25,Unknown,1,1,7,2,...,No,No,No,No,No,No,No,Yes,NO,Physician Referral
3,500364,82442376,Caucasian,Male,35,Unknown,1,1,7,2,...,Up,No,No,No,No,No,Ch,Yes,NO,Physician Referral
4,16680,42519267,Caucasian,Male,45,Unknown,1,1,7,1,...,Steady,No,No,No,No,No,Ch,Yes,NO,Physician Referral


In [43]:
def clean_specific_columns(data):
    """
    Function to clean specific columns in diabetic_data.
    - Standardizes categorical variables.
    - Handles missing or ambiguous entries in 'weight' and 'change'.
    - Standardizes the 'readmitted' column to binary categories.

    Parameters:
    - data (DataFrame): The diabetic_data dataset.

    Returns:
    - DataFrame: Cleaned diabetic_data dataset.
    """
    print("\n--- Cleaning Specific Columns in diabetic_data ---\n")
    
    # 6.1 Standardize categorical variables
    # Capitalize and standardize 'readmitted' and 'change' columns
    data['readmitted'] = data['readmitted'].str.upper().map({'NO': 'Not Readmitted', '>30': 'Readmitted', '<30': 'Readmitted'})
    print("Standardized 'readmitted' to binary categories: 'Not Readmitted' and 'Readmitted'.")
    
    data['change'] = data['change'].map({'No': 'No Change', 'Ch': 'Change'})
    print("Standardized 'change' to: 'No Change' and 'Change'.")

    # 6.2 Review and transform 'weight'
    # Create a binary flag: 'Weight Provided' vs. 'Unknown'
    data['weight'] = data['weight'].replace('?', 'Unknown')
    data['weight_flag'] = data['weight'].apply(lambda x: 'Weight Provided' if x != 'Unknown' else 'Unknown')
    print("Transformed 'weight' to a binary flag: 'Weight Provided' vs. 'Unknown'.")

    return data

def ensure_distinct_descriptions(data):
    """
    Function to ensure all descriptions in IDS_mapping are distinct and meaningful.
    - Removes duplicates.
    - Prints a preview of unique descriptions.

    Parameters:
    - data (DataFrame): The IDS_mapping dataset.

    Returns:
    - DataFrame: Cleaned IDS_mapping dataset.
    """
    print("\n--- Ensuring Distinct Descriptions in IDS_mapping ---\n")
    
    # Check for duplicate descriptions
    duplicate_descriptions = data['description'].duplicated().sum()
    print(f"Found {duplicate_descriptions} duplicate descriptions in 'description'.")
    
    # Drop duplicates if necessary
    data = data.drop_duplicates(subset=['description'])
    print("Ensured all descriptions are distinct and meaningful.")
    
    return data

# Apply cleaning to diabetic_data
diabetes_dataset_cleaned = clean_specific_columns(diabetes_dataset_deduplicated)

# Apply description cleaning to IDS_mapping
IDS_mapping_cleaned = ensure_distinct_descriptions(IDS_mapping_cleaned)

# Preview cleaned diabetic_data
diabetes_dataset_cleaned.head(), IDS_mapping_cleaned.head()


--- Cleaning Specific Columns in diabetic_data ---

Standardized 'readmitted' to binary categories: 'Not Readmitted' and 'Readmitted'.
Standardized 'change' to: 'No Change' and 'Change'.
Transformed 'weight' to a binary flag: 'Weight Provided' vs. 'Unknown'.

--- Ensuring Distinct Descriptions in IDS_mapping ---

Found 5 duplicate descriptions in 'description'.
Ensured all descriptions are distinct and meaningful.


(  encounter_id patient_nbr             race  gender  age   weight  \
 0      2278392     8222157        Caucasian  Female    5  Unknown   
 1       149190    55629189        Caucasian  Female   15  Unknown   
 2        64410    86047875  AfricanAmerican  Female   25  Unknown   
 3       500364    82442376        Caucasian    Male   35  Unknown   
 4        16680    42519267        Caucasian    Male   45  Unknown   
 
    admission_type_id  discharge_disposition_id  admission_source_id  \
 0                  6                        25                    1   
 1                  1                         1                    7   
 2                  1                         1                    7   
 3                  1                         1                    7   
 4                  1                         1                    7   
 
    time_in_hospital  ... glyburide-metformin glipizide-metformin  \
 0                 1  ...                  No                  No   
 1    

In [47]:
import numpy as np

def detect_outliers(data, columns, method='IQR'):
    """
    Function to detect outliers in numerical fields using IQR or Z-score methods.

    Parameters:
    - data (DataFrame): The dataset to analyze.
    - columns (list): List of numerical columns to check for outliers.
    - method (str): Method for outlier detection ('IQR' or 'Z-score').

    Returns:
    - DataFrame: Original data with an added column indicating outliers.
    """
    print(f"\n--- Detecting Outliers using {method} method ---\n")
    
    outlier_flags = pd.DataFrame(index=data.index)

    if method == 'IQR':
        for col in columns:
            Q1 = data[col].quantile(0.25)
            Q3 = data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outlier_flags[col] = (data[col] < lower_bound) | (data[col] > upper_bound)
            print(f"{col}: {outlier_flags[col].sum()} outliers detected.")
    
    elif method == 'Z-score':
        for col in columns:
            mean = data[col].mean()
            std_dev = data[col].std()
            z_scores = (data[col] - mean) / std_dev
            outlier_flags[col] = np.abs(z_scores) > 3  # Z-score threshold
            print(f"{col}: {outlier_flags[col].sum()} outliers detected.")
    
    else:
        raise ValueError("Invalid method. Choose 'IQR' or 'Z-score'.")
    
    # Add a flag column to the original dataset
    data['outlier_flag'] = outlier_flags.any(axis=1)
    return data

# Numerical columns to check for outliers
numerical_columns = ['time_in_hospital', 'num_lab_procedures', 'num_medications']

# Detect outliers using the IQR method
diabetes_dataset_with_outliers = detect_outliers(diabetes_dataset_cleaned, numerical_columns, method='IQR')

# Display a summary of rows flagged as outliers
outliers = diabetes_dataset_with_outliers[diabetes_dataset_with_outliers['outlier_flag']]
outliers_summary = outliers.describe()

import ace_tools as tools; tools.display_dataframe_to_user(name="Summary of Rows with Outliers", dataframe=outliers_summary)

# Preview the dataset with outliers flagged
diabetes_dataset_with_outliers.head()


--- Detecting Outliers using IQR method ---

time_in_hospital: 2252 outliers detected.
num_lab_procedures: 143 outliers detected.
num_medications: 2557 outliers detected.


ModuleNotFoundError: No module named 'ace_tools'

In [49]:
# Display the summary of rows flagged as outliers
print("\n--- Summary of Rows with Outliers ---\n")
print(outliers.describe())

# Display a sample of the rows flagged as outliers
print("\n--- Sample Rows with Outliers ---\n")
print(outliers.head())

# Save the flagged rows to a CSV file for further inspection (optional)
outliers.to_csv("flagged_outliers.csv", index=False)
print("\nFlagged outliers have been saved to 'flagged_outliers.csv'.")


--- Summary of Rows with Outliers ---

               age  admission_type_id  discharge_disposition_id  \
count  4569.000000        4569.000000               4569.000000   
mean     66.405121           2.276866                  5.822062   
std      13.674499           1.404955                  6.376819   
min      15.000000           1.000000                  1.000000   
25%      55.000000           1.000000                  1.000000   
50%      65.000000           2.000000                  3.000000   
75%      75.000000           3.000000                  6.000000   
max      95.000000           8.000000                 28.000000   

       admission_source_id  time_in_hospital  num_lab_procedures  \
count          4569.000000       4569.000000         4569.000000   
mean              4.921865         10.544758           57.717881   
std               4.228767          3.414745           19.319471   
min               1.000000          1.000000            1.000000   
25%             

In [51]:
def validate_consistency(data):
    """
    Function to validate logical consistency across columns in diabetic_data.

    Parameters:
    - data (DataFrame): The dataset to validate.

    Returns:
    - Dictionary: Summary of inconsistencies.
    """
    print("\n--- Validation and Consistency Checks ---\n")
    
    # 9.1 Logical Consistency: admission_type_id and admission_source_id
    print("Checking consistency between 'admission_type_id' and 'admission_source_id'...")
    emergency_admissions = data[(data['admission_type_id'] == 1) & 
                                (~data['admission_source_id'].isin([7, 1]))]
    print(f"Found {len(emergency_admissions)} inconsistent 'emergency' admission types with invalid sources.\n")

    # Logical Consistency: time_in_hospital vs. severity
    print("Checking consistency between 'time_in_hospital' and severity fields...")
    long_stay_low_complexity = data[(data['time_in_hospital'] > 10) & 
                                    (data['num_lab_procedures'] < 10) & 
                                    (data['num_medications'] < 5)]
    print(f"Found {len(long_stay_low_complexity)} cases with long hospital stays but low complexity.\n")

    # 9.2 Mapping Verification
    print("Verifying mappings for numeric fields with descriptions...")
    missing_descriptions = data[data['admission_type_description'].isnull()]
    print(f"Found {len(missing_descriptions)} entries with missing admission type descriptions.\n")
    
    # Create a summary of inconsistencies
    inconsistencies = {
        "Emergency Admissions with Invalid Sources": len(emergency_admissions),
        "Long Stays with Low Complexity": len(long_stay_low_complexity),
        "Missing Admission Type Descriptions": len(missing_descriptions)
    }

    return inconsistencies, emergency_admissions, long_stay_low_complexity, missing_descriptions

# Apply validation and consistency checks
validation_results, invalid_emergency_admissions, invalid_long_stays, missing_descriptions = validate_consistency(diabetes_dataset_with_outliers)

# Display summary of inconsistencies
print("\n--- Summary of Inconsistencies ---\n")
print(validation_results)

# Optional: Save inconsistencies to CSV files for further review
invalid_emergency_admissions.to_csv("invalid_emergency_admissions.csv", index=False)
invalid_long_stays.to_csv("invalid_long_stays.csv", index=False)
missing_descriptions.to_csv("missing_descriptions.csv", index=False)

print("\nInconsistencies have been saved to separate CSV files for further inspection.")



--- Validation and Consistency Checks ---

Checking consistency between 'admission_type_id' and 'admission_source_id'...
Found 3798 inconsistent 'emergency' admission types with invalid sources.

Checking consistency between 'time_in_hospital' and severity fields...
Found 2 cases with long hospital stays but low complexity.

Verifying mappings for numeric fields with descriptions...
Found 0 entries with missing admission type descriptions.


--- Summary of Inconsistencies ---

{'Emergency Admissions with Invalid Sources': 3798, 'Long Stays with Low Complexity': 2, 'Missing Admission Type Descriptions': 0}

Inconsistencies have been saved to separate CSV files for further inspection.


In [53]:
# Analyze Emergency Admission Inconsistencies
print("\n--- Analyzing Emergency Admission Inconsistencies ---\n")
print(f"Total cases with inconsistencies: {len(invalid_emergency_admissions)}")
print("\nSample of Inconsistent Emergency Admissions:")
print(invalid_emergency_admissions.head())

# Analyze the distribution of 'admission_source_id' in inconsistent emergency admissions
emergency_admissions_source_counts = invalid_emergency_admissions['admission_source_id'].value_counts()
print("\nDistribution of 'admission_source_id' in inconsistent emergency admissions:")
print(emergency_admissions_source_counts)

# Analyze Long Stays with Low Complexity
print("\n--- Analyzing Long Stays with Low Complexity ---\n")
print(f"Total cases with inconsistencies: {len(invalid_long_stays)}")
print("\nDetails of Long Stays with Low Complexity:")
print(invalid_long_stays)

# Fix Identified Issues
# For Emergency Admission Inconsistencies:
# Flag these rows for exclusion from modeling
invalid_emergency_admissions['exclude_from_modeling'] = True

# For Long Stays with Low Complexity:
# Flag these rows for further investigation or exclusion
invalid_long_stays['exclude_from_modeling'] = True

# Save flagged datasets for further inspection
invalid_emergency_admissions.to_csv("invalid_emergency_admissions_fixed.csv", index=False)
invalid_long_stays.to_csv("invalid_long_stays_fixed.csv", index=False)

print("\nCorrected files have been saved:")
print("- 'invalid_emergency_admissions_fixed.csv': Corrected emergency admissions.")
print("- 'invalid_long_stays_fixed.csv': Corrected long stays with low complexity.")


--- Analyzing Emergency Admission Inconsistencies ---

Total cases with inconsistencies: 3798

Sample of Inconsistent Emergency Admissions:
    encounter_id patient_nbr       race  gender  age   weight  \
511      4329624    99941634  Caucasian    Male   75  Unknown   
598      4843464     6458139  Caucasian  Female   95  Unknown   
607      4878348    24880779  Caucasian  Female   75  Unknown   
645      5074566    22829274    Unknown    Male   55  Unknown   
747      5698884     1896381  Caucasian  Female   85  Unknown   

     admission_type_id  discharge_disposition_id  admission_source_id  \
511                  1                         1                    4   
598                  1                         6                    5   
607                  1                         3                   20   
645                  1                         1                   20   
747                  1                         3                   20   

     time_in_hospital  ... gl

In [55]:
def investigate_outliers(data, numerical_columns):
    """
    Function to investigate rows with flagged outliers.
    - Identifies inconsistencies in flagged rows.
    - Analyzes potential issues like long stays with minimal procedures.

    Parameters:
    - data (DataFrame): The dataset with outlier flags.
    - numerical_columns (list): List of numerical columns to analyze for outliers.

    Returns:
    - DataFrame: Rows flagged as potential data inconsistencies.
    """
    print("\n--- Investigating Flagged Outliers ---\n")

    # Focus on rows with outliers flagged
    outliers = data[data['outlier_flag']]

    # Investigate long stays with minimal procedures
    inconsistent_cases = outliers[
        (outliers['time_in_hospital'] > 10) & 
        (outliers['num_lab_procedures'] < 10) & 
        (outliers['num_medications'] < 5)
    ]
    print(f"Found {len(inconsistent_cases)} cases with long hospital stays but minimal procedures/medications.")
    
    return inconsistent_cases

def handle_outliers(data, method="replace", numerical_columns=None):
    """
    Function to handle outliers in the dataset.
    - Corrects or excludes outliers based on the chosen method.

    Parameters:
    - data (DataFrame): The dataset to process.
    - method (str): The handling method ('replace' or 'exclude').
    - numerical_columns (list): List of numerical columns to handle for outliers.

    Returns:
    - DataFrame: Processed dataset with outliers handled.
    """
    print(f"\n--- Handling Outliers with Method: {method} ---\n")
    
    if method == "replace":
        # Replace outliers with median values
        for col in numerical_columns:
            Q1 = data[col].quantile(0.25)
            Q3 = data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            median_value = data[col].median()
            data.loc[(data[col] < lower_bound) | (data[col] > upper_bound), col] = median_value
            print(f"Replaced outliers in '{col}' with median value: {median_value}.")
    elif method == "exclude":
        # Exclude rows with flagged outliers
        data = data[~data['outlier_flag']]
        print(f"Excluded rows flagged as outliers. Remaining rows: {len(data)}.")
    else:
        raise ValueError("Invalid method. Choose 'replace' or 'exclude'.")

    return data

# Investigate flagged outliers for inconsistencies
numerical_columns = ['time_in_hospital', 'num_lab_procedures', 'num_medications']
inconsistent_outliers = investigate_outliers(diabetes_dataset_with_outliers, numerical_columns)

# Handle outliers: Replace with median values
diabetes_dataset_corrected = handle_outliers(diabetes_dataset_with_outliers, method="replace", numerical_columns=numerical_columns)

# Save corrected dataset
diabetes_dataset_corrected.to_csv("diabetes_dataset_corrected.csv", index=False)
print("\nCorrected dataset has been saved as 'diabetes_dataset_corrected.csv'.")


--- Investigating Flagged Outliers ---

Found 0 cases with long hospital stays but minimal procedures/medications.

--- Handling Outliers with Method: replace ---

Replaced outliers in 'time_in_hospital' with median value: 4.0.
Replaced outliers in 'num_lab_procedures' with median value: 44.0.
Replaced outliers in 'num_medications' with median value: 15.0.

Corrected dataset has been saved as 'diabetes_dataset_corrected.csv'.


In [57]:
# Step 10: Export Cleaned Data

# 10.1 Export cleaned datasets
diabetes_dataset_corrected.to_csv("cleaned_diabetic_data.csv", index=False)
IDS_mapping_cleaned.to_csv("cleaned_IDS_mapping.csv", index=False)

print("\nCleaned datasets have been exported:")
print("- 'cleaned_diabetic_data.csv': Cleaned diabetic_data.")
print("- 'cleaned_IDS_mapping.csv': Cleaned IDS_mapping.")


Cleaned datasets have been exported:
- 'cleaned_diabetic_data.csv': Cleaned diabetic_data.
- 'cleaned_IDS_mapping.csv': Cleaned IDS_mapping.


In [59]:
# 10.2 Document transformations
transformations = """
Data Cleaning and Transformation Summary:

1. Removed duplicates based on 'encounter_id'.
2. Standardized categorical variables:
   - 'readmitted': Mapped to 'Readmitted' and 'Not Readmitted'.
   - 'change': Standardized to 'Change' and 'No Change'.
   - 'weight': Transformed to binary flag: 'Weight Provided' or 'Unknown'.
3. Replaced missing values and standardized 'admission_type_id' using IDS_mapping.
4. Flagged outliers in numerical fields (e.g., 'time_in_hospital') using IQR method.
5. Replaced outlier values with column medians for consistency.
6. Validated logical consistency between:
   - 'admission_type_id' and 'admission_source_id'.
   - 'time_in_hospital' and complexity indicators.
7. Exported cleaned datasets for analysis.
"""

# Save the documentation to a text file
with open("data_cleaning_transformations.txt", "w") as f:
    f.write(transformations)

print("\nTransformation documentation has been saved as 'data_cleaning_transformations.txt'.")


Transformation documentation has been saved as 'data_cleaning_transformations.txt'.
